In [1]:
import pandas as pd 
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scikeras.wrappers import KerasClassifier

In [2]:
data = pd.read_csv('Churn_Modelling.csv')

data.drop(columns = {'RowNumber','CustomerId','Surname'},inplace = True)

# Gender column -> we can solve by LabelEncoder 
# Geography column-> we can solve by OneHotEncoder


label_encoder = LabelEncoder()
data['Gender'] = label_encoder.fit_transform(data['Gender'])

onehot_encoder = OneHotEncoder(sparse_output=False)
encoded = onehot_encoder.fit_transform(data[['Geography']])
encoded = pd.DataFrame(encoded,columns = onehot_encoder.get_feature_names_out(['Geography']))
encoded.rename(columns = {'Geography_France':'France', 'Geography_Germany':'Germany', 'Geography_Spain':'Spain'},inplace = True)
data = pd.concat([data,encoded],axis = 1)

with open('gender.pkl','wb') as file:
    pickle.dump(label_encoder,file)
with open('geography.pkl','wb') as file:
    pickle.dump(onehot_encoder,file)
feature = data.drop(columns = ['Exited','Geography'],axis = 1)
target = data['Exited']

# Splitting of the data 
X_train,X_test,y_train,y_test = train_test_split(feature,target,train_size=0.8,random_state=42)

# We will use the scalar for giving the value equal weightage
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [3]:

def create_model(layer,neurons):
    model = Sequential()
    model.add(Dense(neurons,activation = 'relu',input_shape = (X_train.shape[1],)))
    
    for _ in range(layer - 1):
        model.add(Dense(neurons,activation = 'relu'))
    
    model.add(Dense(1,activation = 'sigmoid'))
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model
    
    

In [4]:
cls = KerasClassifier(build_fn=create_model,epochs = 50, batch_size = 10,verbose = 0,)

In [5]:
'''from sklearn.model_selection import GridSearchCV
parameters = [{
    'model__layer' : [1,2,3],
    'model__neurons' : [8,16,24,32,64],
    'batch_size' : [10,20]
}]

gridsearch = GridSearchCV(estimator=cls, param_grid=parameters, cv = 5, verbose = 1, n_jobs=-1)
gridsearch.fit(X_train, y_train)

print(f"best accuracy {gridsearch.best_score_} with best parameters {gridsearch.best_params_}") '''
# best accuracy 0.8605 with best parameters {'batch_size': 10, 'model__layer': 1, 'model__neurons': 16} 

'from sklearn.model_selection import GridSearchCV\nparameters = [{\n    \'model__layer\' : [1,2,3],\n    \'model__neurons\' : [8,16,24,32,64],\n    \'batch_size\' : [10,20]\n}]\n\ngridsearch = GridSearchCV(estimator=cls, param_grid=parameters, cv = 5, verbose = 1, n_jobs=-1)\ngridsearch.fit(X_train, y_train)\n\nprint(f"best accuracy {gridsearch.best_score_} with best parameters {gridsearch.best_params_}") '

In [8]:
model = Sequential()
model.add(Dense(16,activation ='relu',input_shape = (X_train.shape[1],)))
model.add(Dense(1,activation = 'sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics = ['accuracy'])

c:\Git Repository\ANNClassifier\ANNenv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
model.fit(X_train,y_train, validation_data = [X_test,y_test], epochs=100, batch_size= 10)

Epoch 1/100


800/800 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7900 - loss: 0.4747 - val_accuracy: 0.8210 - val_loss: 0.4195
Epoch 2/100
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8142 - loss: 0.4218 - val_accuracy: 0.8270 - val_loss: 0.3978
Epoch 3/100
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8275 - loss: 0.3984 - val_accuracy: 0.8390 - val_loss: 0.3783
Epoch 4/100
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8415 - loss: 0.3780 - val_accuracy: 0.8510 - val_loss: 0.3620
Epoch 5/100
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8499 - loss: 0.3648 - val_accuracy: 0.8595 - val_loss: 0.3529
Epoch 6/100
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8547 - loss: 0.3566 - val_accuracy: 0.8540 - val_loss: 0.3501
Epoch 7/100
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8568 - loss: 0.3514 - val_accuracy: 0.8580 - val_loss: 0.3486
Epoch 8/100
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8580 - loss: 0.3478 - val_accuracy: 0.8565